In [82]:
# Extracting Top Solar Flares Tables (BeautifulSoup)

import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

In [88]:
class SpaceWeatherScraper:
    def __init__(self):
        self.base_url = "https://www.spaceweatherlive.com/en/solar-activity/top-50-solar-flares/year/"
        self.headers = {'User-Agent': 'Mozilla/5.0'}

    def get_data(self, years=[1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005,
                              2006, 2007, 2008, 2009, 2010, 2011,
                             2012, 2013, 2014, 2015,
                              2016, 2017, 2018, 2019, 2020,
                               2021, 2022, 2023, 2024, 2025]):
        all_data = []
        for year in years:
            url = f"{self.base_url}{year}"
            response = requests.get(url, headers=self.headers)
            if response.status_code == 200:
                # pd.read_html is the most efficient way to grab <table> tags
                tables = pd.read_html(response.text)
                if tables:
                    df = tables[0]
                    df['Year'] = year
                    all_data.append(df)
        return pd.concat(all_data, ignore_index=True)

In [89]:
class NASACDAWScraper:
    def __init__(self):
        self.url = "https://cdaw.gsfc.nasa.gov/CME_list/radio/waves_type2.html"

    def get_data(self):
        res = requests.get(self.url)
        soup = BeautifulSoup(res.text, 'html.parser')
        
        # NASA stores this specific list inside a <pre> tag
        raw_content = soup.find('pre').text
        lines = raw_content.split('\n')[12:] # Skip header lines
        
        data = []
        for line in lines:
            parts = line.split()
            if len(parts) >= 10:
                data.append({
                    "date": parts[0], "start_time": parts[1],
                    "end_date": parts[2], "speed": parts[7]
                })
        return pd.DataFrame(data)

In [90]:
# 1. Get Top Flares (Labels)
sw_data = SpaceWeatherScraper().get_data()

# 3. Get CME context
nasa_data = NASACDAWScraper().get_data()

print(f"Collected {len(noaa_data)} NOAA rows and {len(nasa_data)} NASA rows.")

C:\Users\jyusu\AppData\Local\Temp\ipykernel_5688\3200609869.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)
C:\Users\jyusu\AppData\Local\Temp\ipykernel_5688\3200609869.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)
C:\Users\jyusu\AppData\Local\Temp\ipykernel_5688\3200609869.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)
C:\Users\jyusu\AppData\Local\Temp\ipykernel_5688\3200609869.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To

Collected 0 NOAA rows and 704 NASA rows.


In [91]:
data_path = r'C:\Solar-Flare-Forecast\Data\Scraped_Data'

os.makedirs(data_path, exist_ok=True)

file_path = os.path.join(data_path, 'sw_data.csv')

# Save
sw_data.to_csv(file_path, index=False)

In [92]:
data_path = r'C:\Solar-Flare-Forecast\Data\Scraped_Data'

os.makedirs(data_path, exist_ok=True)

file_path = os.path.join(data_path, 'nasa_data.csv')

# Save
nasa_data.to_csv(file_path, index=False)